## Try to resample each pulse to 100 samples


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib as mpl
import os
import glob
mpl.rcParams["lines.linewidth"] = 0.96
%matplotlib qt

from ppg_fiducials import *

In [57]:
HOM_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/HOM/'
# data_path = "/Volumes/Untitled/Hom3ostasis/CinC/Data_24hPulses/baseline/"
data_path = "/Volumes/Untitled/Hom3ostasis/CinC/Data_24hPulses/mod/"
sub_names = sorted([f for f in os.listdir(data_path) if not f.startswith('.')])
len(sub_names)

39

In [58]:
sub_ID = sub_names[0]
sub_ID

'10_AC'

In [59]:
data_path

'/Volumes/Untitled/Hom3ostasis/CinC/Data_24hPulses/mod/'

In [60]:
ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]
acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
ppg = pd.read_csv(ppg_path, header = None)[2:].to_numpy().squeeze()
# acc = pd.read_csv(acc_path, header = None)[2:].to_numpy().squeeze()
y_basic = pd.read_csv(data_path + sub_ID + '/' + '/y_basic.csv', header = None).to_numpy().squeeze()
y_diagnostic = pd.read_csv(data_path + sub_ID + '/' + '/y_diagnostic.csv', header = None).to_numpy().squeeze()
I_foot = pd.read_csv(data_path + sub_ID + '/' + '/I_foot.csv', header=None).to_numpy().squeeze() - 1 # -1 because matlab is 1-indexed
I_peak = pd.read_csv(data_path + sub_ID + '/' + '/I_peak.csv', header=None).to_numpy().squeeze() - 1
if I_peak[0] < I_foot[0]:
    I_peak = I_peak[1:]
if I_foot[-1] > I_peak[-1]:
    I_foot = I_foot[:-1]

In [63]:
np.sum(y_diagnostic == 2)

1221

In [66]:
plt.figure(figsize = (10, 5))
plt.plot(ppg)
for i in range(y_basic.shape[0]):
    if y_basic[i] == 1:
        plt.plot(np.arange(I_foot[i],I_foot[i+1]), ppg[np.arange(I_foot[i],I_foot[i+1])], 'r')

In [35]:
# Segment the single pulses
ppg_pulses = [ppg[I_foot[i]:I_foot[i+1]] for i in range(len(I_foot)-1)]
len_pulses = [len(pulse) for pulse in ppg_pulses]

# Interpolate each pulse to 100 samples with cubic spline
ppg_pulses_interp = [np.interp(np.linspace(0, 1, 100), np.linspace(0, 1, len(pulse)), pulse) for pulse in ppg_pulses]
# offset_I_foot = [100 - l for l in len_pulses]
# I_foot_int = I_foot[:-1] + offset_I_foot

# Reconstruct the overall ppg using the interpolated pulses
I_foot_int = np.concatenate([[0], np.cumsum([len(pulse) for pulse in ppg_pulses_interp])])
I_peak_int = [np.argmax(ppg_pulses_interp[i]) + I_foot_int[i] for i in range(len(ppg_pulses_interp))]
ppg_int = np.concatenate(ppg_pulses_interp)
# plt.figure()
# plt.plot(ppg_int)
# plt.plot(I_foot_int[:-1], ppg_int[I_foot_int[:-1]], 'ro')
# plt.plot(I_peak_int, ppg_int[I_peak_int], 'go')

In [34]:
len(I_foot_int), len(I_peak_int)

(105142, 105141)

In [39]:
dn = get_dicrotic_notch(ppg_int, 64, np.array(I_peak_int), list(I_foot_int[:-1]))
dn_diag = np.array(dn)[y_diagnostic[:-1] == 1]

/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:51: RuntimeWarning: divide by zero encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:51: RuntimeWarning: invalid value encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:61: RuntimeWarning: divide by zero encountered in scalar divide
  tau_wmax=(t_w-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:65: RuntimeWarning: invalid value encountered in scalar subtract
  alfa=(beta*tau_wmax-2*tau_wmax+1)/(1-tau_wmax)


In [ ]:
apg_fp = get_apg_fiducials(ppg_int, list(I_foot), I_peak)
e_diag = apg_fp["e"][y_diagnostic == 1].values.astype(int)
a_diag = apg_fp["a"][y_diagnostic == 1].values.astype(int)
b_diag = apg_fp["b"][y_diagnostic == 1].values.astype(int)

vpg_fp = get_vpg_fiducials(ppg_int, list(I_foot))
u_diag = vpg_fp["u"][y_diagnostic == 1].values.astype(int)
v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)
dp_diag = get_diastolic_peak(ppg_int, diag_pulses, dn_diag, e_diag)
fiducials_diag = pd.DataFrame({"onset": diag_pulses[::2], "offset": diag_pulses[1::2]-1, "sp": I_peak_diag, "dn": dn_diag, "dp": dp_diag, "u": u_diag, "v": v_diag, "a": a_diag, "b": b_diag})
fiducials_diag.dropna(inplace=True)
fiducials_diag = fiducials_diag.astype(int)
fiducials_diag.head()

In [40]:
plt.figure()
plt.plot(ppg_int)
plt.plot(dn_diag, ppg_int[dn_diag], 'g*', markersize=6)
for i in range(y_diagnostic.shape[0]):
    if y_diagnostic[i] == 1:
        plt.plot(np.arange(I_foot_int[i],I_foot_int[i+1]), ppg_int[np.arange(I_foot_int[i],I_foot_int[i+1])], 'r')

## Analysis on 24h

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib as mpl
import os
mpl.rcParams["lines.linewidth"] = 0.96
%matplotlib qt

from ppg_fiducials import *

In [7]:
# create a folder for each sub name
data_path =  '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Data_24hPulses/followup'
for sub in sub_names:
    sub_path = os.path.join(data_path, sub)
    if not os.path.exists(sub_path):
        os.makedirs(sub_path)

In [45]:
HOM_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/HOM_mod/'
# data_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Data_24hPulses/baseline/'
sub_names = sorted([f for f in os.listdir(HOM_path) if not f.startswith('.')])
len(sub_names)

39

In [46]:
# make a folder for each sub inside path
path = "/Volumes/Untitled/Hom3ostasis/CinC/Data_24hPulses/mod/"

for sub in sub_names:
    sub_path = os.path.join(path, sub)
    if not os.path.exists(sub_path):
        os.makedirs(sub_path)

In [3]:
import glob

glob.glob(HOM_path + '10_AC/' + '**/BVP.csv', recursive=True)[0]

'/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/HOM/10_AC/1574158703_A01C3E/BVP.csv'

In [58]:
perc_pulses_basic = {} 
perc_pulses_diagnostic = {} 
n_pulse_basic = {} 
n_pulse_diagnostic = {} 
for i, sub_ID in enumerate(sub_names): 
    print(sub_ID)
    perc_pulses_basic[sub_ID] = {}
    perc_pulses_diagnostic[sub_ID] = {}
    n_pulse_basic[sub_ID] = {}
    n_pulse_diagnostic[sub_ID] = {}
    # ppg_path = glob.glob(HOM_path + '10_AC/' + '**/BVP.csv', recursive=True)[0]
    # acc_path = glob.glob(HOM_path + '10_AC/' + '**/ACC.csv', recursive=True)[0]
    # ppg = pd.read_csv(ppg_path, header = None)[2:].to_numpy().squeeze()
    # acc = pd.read_csv(acc_path, header = None)[2:].to_numpy().squeeze()
    y_basic = pd.read_csv(data_path + sub_ID + '/' + '/y_basic.csv', header = None).to_numpy().squeeze()
    y_diagnostic = pd.read_csv(data_path + sub_ID + '/' + '/y_diagnostic.csv', header = None).to_numpy().squeeze()
    I_foot = pd.read_csv(data_path + sub_ID + '/' + '/I_foot.csv', header=None).to_numpy().squeeze() - 1 # -1 because matlab is 1-indexed
    I_peak = pd.read_csv(data_path + sub_ID + '/' + '/I_peak.csv', header=None).to_numpy().squeeze() - 1
    perc_pulses_basic[sub_ID] = np.sum(y_basic)/len(y_basic)
    n_pulse_basic[sub_ID] = np.sum(y_basic)
    perc_pulses_diagnostic[sub_ID] = len(np.nonzero(y_diagnostic)[0])/len(y_diagnostic)
    n_pulse_diagnostic[sub_ID] = len(np.nonzero(y_diagnostic)[0])
    # plt.figure(figsize=(10,5))
    # plt.plot(ppg)
    # for i in range(y_diagnostic.shape[0]):
    #     if y_diagnostic[i] == 1:
    #         plt.plot(np.arange(I_foot[i],I_foot[i+1]), ppg[np.arange(I_foot[i],I_foot[i+1])], 'r')
    # plt.plot(I_foot, ppg[I_foot], 'k*', markersize=2)
    # plt.plot(I_peak, ppg[I_peak], 'g*', markersize=2)

10_AC
11_EC
12_FN
13_MS
14_AD
15_ER
16_PD
17_EG
18_MV
19_FI
1_ID
20_AC
21_CK
22_CM
23_PP
24_SM
25_LP
26_CB
27_CM
28_GM
29_LT
30_LN
31_FB
32_CD
33_NM
34_AS
35_AA
36_GS
37_IC
38_GC
39_NB
3_JF
40_SQ
4_MF
5_PD
6_SS
7_GM
8_MB
9_BC


In [59]:
import seaborn as sns

perc_pulses_basic_df = pd.DataFrame.from_dict(perc_pulses_basic, orient = "index")
perc_pulses_diagnostic_df = pd.DataFrame.from_dict(perc_pulses_diagnostic, orient = "index")

plt.figure(figsize = (10,6))
sns.stripplot(perc_pulses_basic_df.T*100, size = 6, alpha = 0.6)#, jitter = 0.25)
plt.xticks(rotation=12)
# plt.xticks(size=12)
# plt.yticks(size = 12)
plt.ylabel('Basic Quality Pulses (%)')
# plt.savefig('figures/BasicQualityPulses.png', dpi=300, bbox_inches='tight')

n_pulse_basic_df = pd.DataFrame.from_dict(n_pulse_basic, orient = "index")  
plt.figure(figsize = (12,7))
sns.swarmplot(n_pulse_basic_df.T, size = 6)
plt.xticks(rotation=12)

plt.figure(figsize = (10,6))
sns.stripplot(perc_pulses_diagnostic_df.T*100, size = 6, alpha = 0.8, jitter = 0.25)
plt.xticks(rotation=12)
# plt.xticks(size=12)
# plt.yticks(size = 12)
plt.ylabel('Diagnostic Quality Pulses (%)')
# plt.savefig('figures/DagnosticQualityPulses.png', dpi=300, bbox_inches='tight')

n_pulse_diag_df = pd.DataFrame.from_dict(n_pulse_diagnostic, orient = "index")  
plt.figure(figsize = (12,7))
sns.stripplot(n_pulse_diag_df.T, size = 6)
plt.xticks(rotation=12)

/Users/marcellosicbaldi/anaconda3/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/Users/marcellosicbaldi/anaconda3/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/Users/marcellosicbaldi/anaconda3/lib/python3.10/site-packages/seaborn/_oldcore.py:1075: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)
/Users/marcellosicbaldi/anaconda3/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: Future

([0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38],
 [Text(0, 0, '10_AC'),
  Text(1, 0, '11_EC'),
  Text(2, 0, '12_FN'),
  Text(3, 0, '13_MS'),
  Text(4, 0, '14_AD'),
  Text(5, 0, '15_ER'),
  Text(6, 0, '16_PD'),
  Text(7, 0, '17_EG'),
  Text(8, 0, '18_MV'),
  Text(9, 0, '19_FI'),
  Text(10, 0, '1_ID'),
  Text(11, 0, '20_AC'),
  Text(12, 0, '21_CK'),
  Text(13, 0, '22_CM'),
  Text(14, 0, '23_PP'),
  Text(15, 0, '24_SM'),
  Text(16, 0, '25_LP'),
  Text(17, 0, '26_CB'),
  Text(18, 0, '27_CM'),
  Text(19, 0, '28_GM'),
  Text(20, 0, '29_LT'),
  Text(21, 0, '30_LN'),
  Text(22, 0, '31_FB'),
  Text(23, 0, '32_CD'),
  Text(24, 0, '33_NM'),
  Text(25, 0, '34_AS'),
  Text(26, 0, '35_AA'),
  Text(27, 0, '36_GS'),
  Text(28, 0, '37_IC'),
  Text(29, 0, '38_GC'),
  Text(30, 0, '39_NB'),
  Text(31, 0, '3_JF'),
  Text(32, 0, '40_S

In [44]:
plt.close("all")

In [42]:
# interpolate acc from 32 to 64 Hz
acc_int = np.zeros((acc.shape[0]*2, acc.shape[1]))
for i in range(acc.shape[1]):
    acc_int[:, i] = np.interp(np.arange(0, len(acc), 0.5), np.arange(0, len(acc)), acc[:, i])

print(acc_int.shape, ppg.shape) 

plt.figure(figsize = (18, 10))
plt.subplot(2, 1, 1)
plt.plot(acc_int)
plt.subplot(2, 1, 2, sharex = plt.subplot(2, 1, 1))
plt.plot(ppg)

(6207600, 3) (6207608,)


In [4]:
if I_peak[0] < I_foot[0]:
    I_peak = I_peak[1:]
if I_foot[-1] > I_peak[-1]:
    I_foot = I_foot[:-1]

In [5]:
I_peak_diag = I_peak[:-1][y_diagnostic == 1]
start_diag = I_foot[:-1][y_diagnostic == 1]
end_diag = I_foot[1:][y_diagnostic == 1]
# join the start and end of the diagnostic pulses, and sort them
diag_pulses = np.sort(np.concatenate((start_diag, end_diag)))
diag_pulses[::2].shape, I_peak_diag.shape

((27820,), (27820,))

In [6]:
dn = get_dicrotic_notch(ppg, 64, I_peak, list(I_foot))
dn_diag = np.array(dn)[y_diagnostic == 1]

In [7]:
apg_fp = get_apg_fiducials(ppg, list(I_foot), I_peak)
e_diag = apg_fp["e"][y_diagnostic == 1].values.astype(int)
a_diag = apg_fp["a"][y_diagnostic == 1].values.astype(int)
b_diag = apg_fp["b"][y_diagnostic == 1].values.astype(int)

vpg_fp = get_vpg_fiducials(ppg, list(I_foot))
u_diag = vpg_fp["u"][y_diagnostic == 1].values.astype(int)
v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)

In [8]:
dp_diag = get_diastolic_peak(ppg, diag_pulses, dn_diag, e_diag)

In [18]:
fiducials_diag = pd.DataFrame({"onset": diag_pulses[::2], "offset": diag_pulses[1::2]-1, "sp": I_peak_diag, "dn": dn_diag, "dp": dp_diag, "u": u_diag, "v": v_diag, "a": a_diag, "b": b_diag})

In [19]:
fiducials_diag.dropna(inplace=True)
fiducials_diag = fiducials_diag.astype(int)
fiducials_diag.head()

,onset,offset,sp,dn,dp,u,v,a,b
0,413,463,424,434,441,418,429,417,429
1,464,513,474,484,490,468,478,467,478
2,514,563,525,537,540,518,529,517,529
3,564,614,576,588,591,569,580,568,580
4,875,927,886,898,903,880,907,879,891


In [13]:
plt.figure()
plt.plot(ppg)
plt.plot(fiducials_diag["dp"], ppg[fiducials_diag["dp"]], 'k*', markersize=6)
plt.plot(fiducials_diag["dn"], ppg[fiducials_diag["dn"]], 'g*', markersize=6)
for i in range(y_diagnostic.shape[0]):
    if y_diagnostic[i] == 1:
        plt.plot(np.arange(I_foot[i],I_foot[i+1]), ppg[np.arange(I_foot[i],I_foot[i+1])], 'r')

### Biomarkers extraction

In [20]:
fs = 64

In [21]:
# ----- Time Intervals -----
# Tpi: time between the pulse onset and offset
# Tsys: time between the pulse onset and dn
# Tdia: time between the dn and pulse offset
# Tsp: time between the pulse onset and sp
# Tdp: time between the pulse onset and dp
# dT: time between sp and dp
Tpi = fiducials_diag["offset"]/fs - fiducials_diag["onset"]/fs
Tsys = fiducials_diag["dn"]/fs - fiducials_diag["onset"]/fs
Tdia = fiducials_diag["offset"]/fs - fiducials_diag["dn"]/fs
Tsp = fiducials_diag["sp"]/fs - fiducials_diag["onset"]/fs
Tdp = fiducials_diag["dp"]/fs - fiducials_diag["onset"]/fs
dT = fiducials_diag["dp"]/fs - fiducials_diag["sp"]/fs

# ---- Amplitudes -----
# Asp: difference in amplitude between onset and sp
# Adn: difference in amplitude between onset and dn
# Adp: difference in amplitude between onset and dp
Asp = ppg[fiducials_diag["sp"]] - ppg[fiducials_diag["onset"]]
Adn = ppg[fiducials_diag["dn"]] - ppg[fiducials_diag["onset"]]
Adp = ppg[fiducials_diag["dp"]] - ppg[fiducials_diag["onset"]]

# ---- Areas -----
# AUCpi: area under the whole pulse (between onset and offset)
# AUCsys: area under pulse wave between onset and dn
# AUCdia: area under pulse wave between dn and offset
AUCpi = np.zeros(len(fiducials_diag))
AUCsys = np.zeros(len(fiducials_diag))
AUCdia = np.zeros(len(fiducials_diag))
for i in range(len(fiducials_diag)):
    AUCpi[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["offset"].iloc[i]])
    AUCsys[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["dn"].iloc[i]])
    AUCdia[i] = np.trapz(ppg[fiducials_diag["dn"].iloc[i]:fiducials_diag["offset"].iloc[i]])

# ---- Ratios -----
# TsysTdia: ratio between Tsys and Tdia
# TspTpi: ratio between Tsp and Tpi
# AdpAsp: ratio between Adp and Asp --> reflection index! 
# IPA: ratio of the area under diastolic curve and the area under systolic curve
# TspAsp: ratio between Tsp and Asp
# AspdT: ratio between Asp and dT --> Stifness index
# AspTpiTsp: ratio between Asp and (Tpi - Tsp)
TsysTdia = Tsys/Tdia
TspTpi = Tsp/Tpi
AdpAsp = Adp/Asp
IPA = AUCdia/AUCsys
TspAsp = Tsp/Asp
AspdT = Asp/dT
AspTpiTsp = Asp/(Tpi - Tsp)

# ---- Derivatives -----
# Tu: time between the pulse onset and u
# Tv: time between the pulse onset and v
# Ta: time between the pulse onset and a
# Tb: time between the pulse onset and b
Tu = fiducials_diag["u"]/fs - fiducials_diag["onset"]/fs
Tv = fiducials_diag["v"]/fs - fiducials_diag["onset"]/fs
Ta = fiducials_diag["a"]/fs - fiducials_diag["onset"]/fs
Tb = fiducials_diag["b"]/fs - fiducials_diag["onset"]/fs

features = pd.DataFrame({"Tpi": Tpi, "Tsys": Tsys, "Tdia": Tdia, "Tsp": Tsp, "Tdp": Tdp, "dT": dT, "Asp": Asp, "Adn": Adn, "Adp": Adp, "AUCpi": AUCpi, "AUCsys": AUCsys, "AUCdia": AUCdia, "Tsys/Tdia": TsysTdia, "Tsp/Tpi": TspTpi, "Adp/Asp": AdpAsp, "IPA": IPA, "Tsp/Asp": TspAsp, "Asp/dT": AspdT, "Asp/(Tpi-Tsp)": AspTpiTsp, "Tu": Tu, "Tv": Tv, "Ta": Ta, "Tb": Tb})
features.head()

,Tpi,Tsys,Tdia,Tsp,Tdp,dT,Asp,Adn,Adp,AUCpi,...,Tsp/Tpi,Adp/Asp,IPA,Tsp/Asp,Asp/dT,Asp/(Tpi-Tsp),Tu,Tv,Ta,Tb
0,0.781250,0.328125,0.453125,0.171875,0.437500,0.265625,98.20,66.66,65.80,165.440,...,0.220000,0.670061,-0.491982,0.001750,369.694118,161.148718,0.078125,0.250000,0.062500,0.250000
1,0.765625,0.312500,0.453125,0.156250,0.406250,0.250000,88.88,56.89,56.35,-64.265,...,0.204082,0.634001,-1.302628,0.001758,355.520000,145.854359,0.062500,0.218750,0.046875,0.218750
2,0.765625,0.359375,0.406250,0.171875,0.406250,0.234375,105.31,75.43,75.27,113.150,...,0.224490,0.714747,-0.714057,0.001632,449.322667,177.364211,0.062500,0.234375,0.046875,0.234375
3,0.781250,0.375000,0.406250,0.187500,0.421875,0.234375,111.92,85.65,85.95,135.920,...,0.240000,0.767959,-0.676375,0.001675,477.525333,188.496842,0.078125,0.250000,0.062500,0.250000
4,0.812500,0.359375,0.453125,0.171875,0.437500,0.265625,118.78,98.03,103.31,123.815,...,0.211538,0.869759,-0.576183,0.001447,447.171765,185.412683,0.078125,0.500000,0.062500,0.250000


In [22]:
features.shape

(27744, 23)

In [25]:
# Explore a subset of the features with seaborn
import seaborn as sns

# Plot the temporal patterns of the features
plt.figure(figsize=(18, 10))
sns.lineplot(data=features[["Tpi", "Tsys", "Tdia"]])

<Axes: >

## Biomarkers extraction -- 24 Hours Diag Old No Int

In [9]:
import glob

In [10]:
HOM_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/HOM/'
data_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Data_24hPulses/baseline/'
sub_names = sorted([f for f in os.listdir(data_path) if not f.startswith('.')])
len(sub_names)

39

In [11]:
fs = 64
for i, sub_ID in enumerate(sub_names): 
    print(sub_ID)
    ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]
    acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
    ppg = pd.read_csv(ppg_path, header = None)[2:].to_numpy().squeeze()
    acc = pd.read_csv(acc_path, header = None)[2:].to_numpy().squeeze()
    y_basic = pd.read_csv(data_path + sub_ID + '/' + '/y_basic.csv', header = None).to_numpy().squeeze()
    y_diagnostic = pd.read_csv(data_path + sub_ID + '/' + '/y_diagnostic.csv', header = None).to_numpy().squeeze()
    I_foot = pd.read_csv(data_path + sub_ID + '/' + '/I_foot.csv', header=None).to_numpy().squeeze() - 1 # -1 because matlab is 1-indexed
    I_peak = pd.read_csv(data_path + sub_ID + '/' + '/I_peak.csv', header=None).to_numpy().squeeze() - 1
    if I_peak[0] < I_foot[0]:
        I_peak = I_peak[1:]
    if I_foot[-1] > I_peak[-1]:
        I_foot = I_foot[:-1]
    I_peak_diag = I_peak[:-1][y_diagnostic == 1]
    start_diag = I_foot[:-1][y_diagnostic == 1]
    end_diag = I_foot[1:][y_diagnostic == 1]
    diag_pulses = np.sort(np.concatenate((start_diag, end_diag)))
    diag_pulses[::2].shape, I_peak_diag.shape

    dn = get_dicrotic_notch(ppg, 64, I_peak, list(I_foot))
    dn_diag = np.array(dn)[y_diagnostic == 1]
    apg_fp = get_apg_fiducials(ppg, list(I_foot), I_peak)
    e_diag = apg_fp["e"][y_diagnostic == 1].values.astype(int)
    a_diag = apg_fp["a"][y_diagnostic == 1].values.astype(int)
    b_diag = apg_fp["b"][y_diagnostic == 1].values.astype(int)

    vpg_fp = get_vpg_fiducials(ppg, list(I_foot))
    u_diag = vpg_fp["u"][y_diagnostic == 1].values.astype(int)
    v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)
    dp_diag = get_diastolic_peak(ppg, diag_pulses, dn_diag, e_diag)
    fiducials_diag = pd.DataFrame({"onset": diag_pulses[::2], "offset": diag_pulses[1::2]-1, "sp": I_peak_diag, "dn": dn_diag, "dp": dp_diag, "u": u_diag, "v": v_diag, "a": a_diag, "b": b_diag})
    fiducials_diag[fiducials_diag < -5000000] = np.nan
    fiducials_diag.dropna(inplace=True)
    fiducials_diag = fiducials_diag.astype(int)

    # # plt.figure()
    # # plt.plot(ppg)
    # # plt.plot(fiducials_diag["dp"], ppg[fiducials_diag["dp"]], 'k*', markersize=6)
    # # plt.plot(fiducials_diag["dn"], ppg[fiducials_diag["dn"]], 'g*', markersize=6)
    # # for j in range(y_diagnostic.shape[0]):
    # #     if y_diagnostic[j] == 1:
    # #         plt.plot(np.arange(I_foot[j],I_foot[j+1]), ppg[np.arange(I_foot[j],I_foot[j+1])], 'r')
    # # ----- Time Intervals -----
    # # Tpi: time between the pulse onset and offset
    # # Tsys: time between the pulse onset and dn
    # # Tdia: time between the dn and pulse offset
    # # Tsp: time between the pulse onset and sp
    # # Tdp: time between the pulse onset and dp
    # # dT: time between sp and dp
    # Tpi = fiducials_diag["offset"]/fs - fiducials_diag["onset"]/fs
    # Tsys = fiducials_diag["dn"]/fs - fiducials_diag["onset"]/fs
    # Tdia = fiducials_diag["offset"]/fs - fiducials_diag["dn"]/fs
    # Tsp = fiducials_diag["sp"]/fs - fiducials_diag["onset"]/fs
    # Tdp = fiducials_diag["dp"]/fs - fiducials_diag["onset"]/fs
    # dT = fiducials_diag["dp"]/fs - fiducials_diag["sp"]/fs

    # # ---- Amplitudes -----
    # # Asp: difference in amplitude between onset and sp
    # # Adn: difference in amplitude between onset and dn
    # # Adp: difference in amplitude between onset and dp
    # Asp = ppg[fiducials_diag["sp"]] - ppg[fiducials_diag["onset"]]
    # Adn = ppg[fiducials_diag["dn"]] - ppg[fiducials_diag["onset"]]
    # Adp = ppg[fiducials_diag["dp"]] - ppg[fiducials_diag["onset"]]

    # # ---- Areas -----
    # # AUCpi: area under the whole pulse (between onset and offset)
    # # AUCsys: area under pulse wave between onset and dn
    # # AUCdia: area under pulse wave between dn and offset
    # AUCpi = np.zeros(len(fiducials_diag))
    # AUCsys = np.zeros(len(fiducials_diag))
    # AUCdia = np.zeros(len(fiducials_diag))
    # for i in range(len(fiducials_diag)):
    #     AUCpi[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["offset"].iloc[i]])
    #     AUCsys[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["dn"].iloc[i]])
    #     AUCdia[i] = np.trapz(ppg[fiducials_diag["dn"].iloc[i]:fiducials_diag["offset"].iloc[i]])

    # # ---- Ratios -----
    # # TsysTdia: ratio between Tsys and Tdia
    # # TspTpi: ratio between Tsp and Tpi
    # # AdpAsp: ratio between Adp and Asp --> reflection index! 
    # # IPA: ratio of the area under diastolic curve and the area under systolic curve
    # # TspAsp: ratio between Tsp and Asp
    # # AspdT: ratio between Asp and dT --> Stifness index
    # # AspTpiTsp: ratio between Asp and (Tpi - Tsp)
    # TsysTdia = Tsys/Tdia
    # TspTpi = Tsp/Tpi
    # AdpAsp = Adp/Asp
    # IPA = AUCdia/AUCsys
    # TspAsp = Tsp/Asp
    # AspdT = Asp/dT
    # AspTpiTsp = Asp/(Tpi - Tsp)

    # # ---- Derivatives -----
    # # Tu: time between the pulse onset and u
    # # Tv: time between the pulse onset and v
    # # Ta: time between the pulse onset and a
    # # Tb: time between the pulse onset and b
    # Tu = fiducials_diag["u"]/fs - fiducials_diag["onset"]/fs
    # Tv = fiducials_diag["v"]/fs - fiducials_diag["onset"]/fs
    # Ta = fiducials_diag["a"]/fs - fiducials_diag["onset"]/fs
    # Tb = fiducials_diag["b"]/fs - fiducials_diag["onset"]/fs

    # features = pd.DataFrame({"Tpi": Tpi, "Tsys": Tsys, "Tdia": Tdia, "Tsp": Tsp, "Tdp": Tdp, "dT": dT, "Asp": Asp, "Adn": Adn, "Adp": Adp, "AUCpi": AUCpi, "AUCsys": AUCsys, "AUCdia": AUCdia, "Tsys/Tdia": TsysTdia, "Tsp/Tpi": TspTpi, "Adp/Asp": AdpAsp, "IPA": IPA, "Tsp/Asp": TspAsp, "Asp/dT": AspdT, "Asp/(Tpi-Tsp)": AspTpiTsp, "Tu": Tu, "Tv": Tv, "Ta": Ta, "Tb": Tb})
    # features.to_csv(data_path + sub_ID + '/' + '/features.csv', index = False)
    fiducials_diag.to_csv(data_path + sub_ID + '/' + '/fiducials.csv', index = False)

10_AC


/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1192103018.py:31: RuntimeWarning: invalid value encountered in cast
  v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)


11_EC


/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:51: RuntimeWarning: divide by zero encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:51: RuntimeWarning: invalid value encountered in divide
  tau=(t-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:61: RuntimeWarning: divide by zero encountered in scalar divide
  tau_wmax=(t_w-t_Pmax)/(T_beat-t_Pmax)
/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Code/ppg_fiducials.py:65: RuntimeWarning: invalid value encountered in scalar subtract
  alfa=(beta*tau_wmax-2*tau_wmax+1)/(1-tau_wmax)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv144000

12_FN
13_MS


...

/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1192103018.py:31: RuntimeWarning: invalid value encountered in cast
  v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)


8_MB


/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1192103018.py:31: RuntimeWarning: invalid value encountered in cast
  v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)


9_BC


/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1192103018.py:25: RuntimeWarning: invalid value encountered in cast
  e_diag = apg_fp["e"][y_diagnostic == 1].values.astype(int)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1192103018.py:26: RuntimeWarning: invalid value encountered in cast
  a_diag = apg_fp["a"][y_diagnostic == 1].values.astype(int)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1192103018.py:27: RuntimeWarning: invalid value encountered in cast
  b_diag = apg_fp["b"][y_diagnostic == 1].values.astype(int)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1192103018.py:31: RuntimeWarning: invalid value encountered in cast
  v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)


In [12]:
HOM_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/HOM2/'
data_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Data_24hPulses/followup/'
sub_names = sorted([f for f in os.listdir(data_path) if not f.startswith('.')])
len(sub_names)

39

In [13]:
fs = 64
for i, sub_ID in enumerate(sub_names): 
    print(sub_ID)
    ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]
    acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
    ppg = pd.read_csv(ppg_path, header = None)[2:].to_numpy().squeeze()
    acc = pd.read_csv(acc_path, header = None)[2:].to_numpy().squeeze()
    y_basic = pd.read_csv(data_path + sub_ID + '/' + '/y_basic.csv', header = None).to_numpy().squeeze()
    y_diagnostic = pd.read_csv(data_path + sub_ID + '/' + '/y_diagnostic.csv', header = None).to_numpy().squeeze()
    I_foot = pd.read_csv(data_path + sub_ID + '/' + '/I_foot.csv', header=None).to_numpy().squeeze() - 1 # -1 because matlab is 1-indexed
    I_peak = pd.read_csv(data_path + sub_ID + '/' + '/I_peak.csv', header=None).to_numpy().squeeze() - 1
    if I_peak[0] < I_foot[0]:
        I_peak = I_peak[1:]
    if I_foot[-1] > I_peak[-1]:
        I_foot = I_foot[:-1]
    I_peak_diag = I_peak[:-1][y_diagnostic == 1]
    start_diag = I_foot[:-1][y_diagnostic == 1]
    end_diag = I_foot[1:][y_diagnostic == 1]
    diag_pulses = np.sort(np.concatenate((start_diag, end_diag)))
    diag_pulses[::2].shape, I_peak_diag.shape

    dn = get_dicrotic_notch(ppg, 64, I_peak, list(I_foot))
    dn_diag = np.array(dn)[y_diagnostic == 1]
    apg_fp = get_apg_fiducials(ppg, list(I_foot), I_peak)
    e_diag = apg_fp["e"][y_diagnostic == 1].values.astype(int)
    a_diag = apg_fp["a"][y_diagnostic == 1].values.astype(int)
    b_diag = apg_fp["b"][y_diagnostic == 1].values.astype(int)

    vpg_fp = get_vpg_fiducials(ppg, list(I_foot))
    u_diag = vpg_fp["u"][y_diagnostic == 1].values.astype(int)
    v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)
    dp_diag = get_diastolic_peak(ppg, diag_pulses, dn_diag, e_diag)
    fiducials_diag = pd.DataFrame({"onset": diag_pulses[::2], "offset": diag_pulses[1::2]-1, "sp": I_peak_diag, "dn": dn_diag, "dp": dp_diag, "u": u_diag, "v": v_diag, "a": a_diag, "b": b_diag})
    fiducials_diag[fiducials_diag < -5000000] = np.nan
    fiducials_diag.dropna(inplace=True)
    fiducials_diag = fiducials_diag.astype(int)

    # plt.figure()
    # plt.plot(ppg)
    # plt.plot(fiducials_diag["dp"], ppg[fiducials_diag["dp"]], 'k*', markersize=6)
    # plt.plot(fiducials_diag["dn"], ppg[fiducials_diag["dn"]], 'g*', markersize=6)
    # for j in range(y_diagnostic.shape[0]):
    #     if y_diagnostic[j] == 1:
    #         plt.plot(np.arange(I_foot[j],I_foot[j+1]), ppg[np.arange(I_foot[j],I_foot[j+1])], 'r')
    # ----- Time Intervals -----
    # Tpi: time between the pulse onset and offset
    # Tsys: time between the pulse onset and dn
    # Tdia: time between the dn and pulse offset
    # Tsp: time between the pulse onset and sp
    # Tdp: time between the pulse onset and dp
    # dT: time between sp and dp
    Tpi = fiducials_diag["offset"]/fs - fiducials_diag["onset"]/fs
    Tsys = fiducials_diag["dn"]/fs - fiducials_diag["onset"]/fs
    Tdia = fiducials_diag["offset"]/fs - fiducials_diag["dn"]/fs
    Tsp = fiducials_diag["sp"]/fs - fiducials_diag["onset"]/fs
    Tdp = fiducials_diag["dp"]/fs - fiducials_diag["onset"]/fs
    dT = fiducials_diag["dp"]/fs - fiducials_diag["sp"]/fs

    # ---- Amplitudes -----
    # Asp: difference in amplitude between onset and sp
    # Adn: difference in amplitude between onset and dn
    # Adp: difference in amplitude between onset and dp
    Asp = ppg[fiducials_diag["sp"]] - ppg[fiducials_diag["onset"]]
    Adn = ppg[fiducials_diag["dn"]] - ppg[fiducials_diag["onset"]]
    Adp = ppg[fiducials_diag["dp"]] - ppg[fiducials_diag["onset"]]

    # ---- Areas -----
    # AUCpi: area under the whole pulse (between onset and offset)
    # AUCsys: area under pulse wave between onset and dn
    # AUCdia: area under pulse wave between dn and offset
    AUCpi = np.zeros(len(fiducials_diag))
    AUCsys = np.zeros(len(fiducials_diag))
    AUCdia = np.zeros(len(fiducials_diag))
    for i in range(len(fiducials_diag)):
        AUCpi[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["offset"].iloc[i]])
        AUCsys[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["dn"].iloc[i]])
        AUCdia[i] = np.trapz(ppg[fiducials_diag["dn"].iloc[i]:fiducials_diag["offset"].iloc[i]])

    # ---- Ratios -----
    # TsysTdia: ratio between Tsys and Tdia
    # TspTpi: ratio between Tsp and Tpi
    # AdpAsp: ratio between Adp and Asp --> reflection index! 
    # IPA: ratio of the area under diastolic curve and the area under systolic curve
    # TspAsp: ratio between Tsp and Asp
    # AspdT: ratio between Asp and dT --> Stifness index
    # AspTpiTsp: ratio between Asp and (Tpi - Tsp)
    TsysTdia = Tsys/Tdia
    TspTpi = Tsp/Tpi
    AdpAsp = Adp/Asp
    IPA = AUCdia/AUCsys
    TspAsp = Tsp/Asp
    AspdT = Asp/dT
    AspTpiTsp = Asp/(Tpi - Tsp)

    # ---- Derivatives -----
    # Tu: time between the pulse onset and u
    # Tv: time between the pulse onset and v
    # Ta: time between the pulse onset and a
    # Tb: time between the pulse onset and b
    Tu = fiducials_diag["u"]/fs - fiducials_diag["onset"]/fs
    Tv = fiducials_diag["v"]/fs - fiducials_diag["onset"]/fs
    Ta = fiducials_diag["a"]/fs - fiducials_diag["onset"]/fs
    Tb = fiducials_diag["b"]/fs - fiducials_diag["onset"]/fs

    features = pd.DataFrame({"Tpi": Tpi, "Tsys": Tsys, "Tdia": Tdia, "Tsp": Tsp, "Tdp": Tdp, "dT": dT, "Asp": Asp, "Adn": Adn, "Adp": Adp, "AUCpi": AUCpi, "AUCsys": AUCsys, "AUCdia": AUCdia, "Tsys/Tdia": TsysTdia, "Tsp/Tpi": TspTpi, "Adp/Asp": AdpAsp, "IPA": IPA, "Tsp/Asp": TspAsp, "Asp/dT": AspdT, "Asp/(Tpi-Tsp)": AspTpiTsp, "Tu": Tu, "Tv": Tv, "Ta": Ta, "Tb": Tb})
    features.to_csv(data_path + sub_ID + '/' + '/features.csv', index = False)
    fiducials_diag.to_csv(data_path + sub_ID + '/' + '/fiducials.csv', index = False)

10_AC
11_EC
12_FN


/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:25: RuntimeWarning: invalid value encountered in cast
  e_diag = apg_fp["e"][y_diagnostic == 1].values.astype(int)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:26: RuntimeWarning: invalid value encountered in cast
  a_diag = apg_fp["a"][y_diagnostic == 1].values.astype(int)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:27: RuntimeWarning: invalid value encountered in cast
  b_diag = apg_fp["b"][y_diagnostic == 1].values.astype(int)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:31: RuntimeWarning: invalid value encountered in cast
  v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)


13_MS


/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:31: RuntimeWarning: invalid value encountered in cast
  v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)


14_AD
15_ER


...

/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:31: RuntimeWarning: invalid value encountered in cast
  v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)


8_MB


/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:31: RuntimeWarning: invalid value encountered in cast
  v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)


9_BC


/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:25: RuntimeWarning: invalid value encountered in cast
  e_diag = apg_fp["e"][y_diagnostic == 1].values.astype(int)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:26: RuntimeWarning: invalid value encountered in cast
  a_diag = apg_fp["a"][y_diagnostic == 1].values.astype(int)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:27: RuntimeWarning: invalid value encountered in cast
  b_diag = apg_fp["b"][y_diagnostic == 1].values.astype(int)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:31: RuntimeWarning: invalid value encountered in cast
  v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)


In [ ]:
plt.close("all")

In [51]:
fiducials_diag[fiducials_diag < -5000000] = np.nan
fiducials_diag.dropna(inplace=True)

## Biomarkers extraction -- 24 hours DiagNew

In [5]:
import glob

HOM_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/HOM_mod/'
data_path = '/Volumes/Untitled/Hom3ostasis/CinC/Data_24hPulses/mod_newQuality/'
sub_names = sorted([f for f in os.listdir(data_path) if not f.startswith('.')])
len(sub_names)

39

In [38]:
for i, sub_ID in enumerate(sub_names):
    os.mkdir(data_path + sub_ID + '/output_GGIR')

In [4]:
from load_utils import load_csv

In [8]:
fs = 64
for i, sub_ID in enumerate(sub_names): 
    sub_ID = "7_GM"
    print(sub_ID)
    # ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]
    acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]

    # ppg = pd.read_csv(ppg_path, header = None)#[2:].to_numpy().squeeze()

    acc = load_csv(acc_path, E4=True)

    acc.to_csv(data_path + sub_ID + '/input_GGIR' + '/acc.csv', index_label="time")
    break
    # acc = pd.read_csv(acc_path, header = None)[2:].to_numpy().squeeze()
    y_basic = pd.read_csv(data_path + sub_ID + '/' + '/y_basic.csv', header = None).to_numpy().squeeze()
    y_diagnostic = pd.read_csv(data_path + sub_ID + '/' + '/y_diagnostic.csv', header = None).to_numpy().squeeze()
    I_foot = pd.read_csv(data_path + sub_ID + '/' + '/I_foot.csv', header=None).to_numpy().squeeze() - 1 # -1 because matlab is 1-indexed
    I_peak = pd.read_csv(data_path + sub_ID + '/' + '/I_peak.csv', header=None).to_numpy().squeeze() - 1
    if I_peak[0] < I_foot[0]:
        I_peak = I_peak[1:]
    if I_foot[-1] > I_peak[-1]:
        I_foot = I_foot[:-1]

    ppg_pulses = [ppg[I_foot[i]:I_foot[i+1]] for i in range(len(I_foot)-1)]

    # Interpolate each pulse to 100 samples with cubic spline
    ppg_pulses_interp = [np.interp(np.linspace(0, 1, 100), np.linspace(0, 1, len(pulse)), pulse) for pulse in ppg_pulses]

    # Reconstruct the overall ppg using the interpolated pulses
    I_foot = np.concatenate([[0], np.cumsum([len(pulse) for pulse in ppg_pulses_interp])])
    I_peak = np.array([np.argmax(ppg_pulses_interp[i]) + I_foot[i] for i in range(len(ppg_pulses_interp))])
    ppg = np.concatenate(ppg_pulses_interp)

    I_peak_diag = I_peak[y_diagnostic == 2]
    start_diag = I_foot[:-1][y_diagnostic == 2]
    end_diag = I_foot[1:][y_diagnostic == 2]
    diag_pulses = np.sort(np.concatenate((start_diag, end_diag)))
    diag_pulses[::2].shape, I_peak_diag.shape

    dn = get_dicrotic_notch(ppg, 64, I_peak, list(I_foot))
    dn_diag = np.array(dn)[y_diagnostic == 2]
    apg_fp = get_apg_fiducials(ppg, list(I_foot), I_peak)
    e_diag = apg_fp["e"][y_diagnostic == 2].values.astype(int)
    a_diag = apg_fp["a"][y_diagnostic == 2].values.astype(int)
    b_diag = apg_fp["b"][y_diagnostic == 2].values.astype(int)

    vpg_fp = get_vpg_fiducials(ppg, list(I_foot))
    u_diag = vpg_fp["u"][y_diagnostic == 2].values.astype(int)
    v_diag = vpg_fp["v"][y_diagnostic == 2].values.astype(int)
    dp_diag = get_diastolic_peak(ppg, diag_pulses, dn_diag, e_diag)
    fiducials_diag = pd.DataFrame({"onset": diag_pulses[::2], "offset": diag_pulses[1::2]-1, "sp": I_peak_diag, "dn": dn_diag, "dp": dp_diag, "u": u_diag, "v": v_diag, "a": a_diag, "b": b_diag})
    fiducials_diag[fiducials_diag < -5000000] = np.nan
    fiducials_diag.dropna(inplace=True)
    fiducials_diag = fiducials_diag.astype(int)

    # plt.figure()
    # plt.plot(ppg)
    # plt.plot(fiducials_diag["dp"], ppg[fiducials_diag["dp"]], 'k*', markersize=6)
    # plt.plot(fiducials_diag["dn"], ppg[fiducials_diag["dn"]], 'g*', markersize=6)
    # for j in range(y_diagnostic.shape[0]):
    #     if y_diagnostic[j] == 1:
    #         plt.plot(np.arange(I_foot[j],I_foot[j+1]), ppg[np.arange(I_foot[j],I_foot[j+1])], 'r')
    # ----- Time Intervals -----
    # Tpi: time between the pulse onset and offset
    # Tsys: time between the pulse onset and dn
    # Tdia: time between the dn and pulse offset
    # Tsp: time between the pulse onset and sp
    # Tdp: time between the pulse onset and dp
    # dT: time between sp and dp
    Tpi = fiducials_diag["offset"] - fiducials_diag["onset"]
    Tsys = fiducials_diag["dn"] - fiducials_diag["onset"]
    Tdia = fiducials_diag["offset"] - fiducials_diag["dn"]
    Tsp = fiducials_diag["sp"] - fiducials_diag["onset"]
    Tdp = fiducials_diag["dp"] - fiducials_diag["onset"]
    dT = fiducials_diag["dp"] - fiducials_diag["sp"]

    # ---- Amplitudes -----
    # Asp: difference in amplitude between onset and sp
    # Adn: difference in amplitude between onset and dn
    # Adp: difference in amplitude between onset and dp
    Asp = ppg[fiducials_diag["sp"]] - ppg[fiducials_diag["onset"]]
    Adn = ppg[fiducials_diag["dn"]] - ppg[fiducials_diag["onset"]]
    Adp = ppg[fiducials_diag["dp"]] - ppg[fiducials_diag["onset"]]

    # ---- Areas -----
    # AUCpi: area under the whole pulse (between onset and offset)
    # AUCsys: area under pulse wave between onset and dn
    # AUCdia: area under pulse wave between dn and offset
    AUCpi = np.zeros(len(fiducials_diag))
    AUCsys = np.zeros(len(fiducials_diag))
    AUCdia = np.zeros(len(fiducials_diag))
    for i in range(len(fiducials_diag)):
        AUCpi[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["offset"].iloc[i]])
        AUCsys[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["dn"].iloc[i]])
        AUCdia[i] = np.trapz(ppg[fiducials_diag["dn"].iloc[i]:fiducials_diag["offset"].iloc[i]])

    # ---- Ratios -----
    # TsysTdia: ratio between Tsys and Tdia
    # TspTpi: ratio between Tsp and Tpi
    # AdpAsp: ratio between Adp and Asp --> reflection index! 
    # IPA: ratio of the area under diastolic curve and the area under systolic curve
    # TspAsp: ratio between Tsp and Asp
    # AspdT: ratio between Asp and dT --> Stifness index
    # AspTpiTsp: ratio between Asp and (Tpi - Tsp)
    TsysTdia = Tsys/Tdia
    TspTpi = Tsp/Tpi
    AdpAsp = Adp/Asp
    IPA = AUCdia/AUCsys
    TspAsp = Tsp/Asp
    AspdT = Asp/dT
    AspTpiTsp = Asp/(Tpi - Tsp)
    Rslope = Asp/Tsys

    # ---- Derivatives -----
    # Tu: time between the pulse onset and u
    # Tv: time between the pulse onset and v
    # Ta: time between the pulse onset and a
    # Tb: time between the pulse onset and b
    Tu = fiducials_diag["u"] - fiducials_diag["onset"]
    Tv = fiducials_diag["v"] - fiducials_diag["onset"]
    Ta = fiducials_diag["a"] - fiducials_diag["onset"]
    Tb = fiducials_diag["b"] - fiducials_diag["onset"]

    features = pd.DataFrame({"Tpi": Tpi, "Tsys": Tsys, "Tdia": Tdia, "Tsp": Tsp, "Tdp": Tdp, "dT": dT, "Asp": Asp, "Adn": Adn, "Adp": Adp, "AUCpi": AUCpi, "AUCsys": AUCsys, "AUCdia": AUCdia, "Tsys/Tdia": TsysTdia, "Tsp/Tpi": TspTpi, "Adp/Asp": AdpAsp, "IPA": IPA, "Tsp/Asp": TspAsp, "Asp/dT": AspdT, "Asp/(Tpi-Tsp)": AspTpiTsp, "Rslope": Rslope, "Tu": Tu, "Tv": Tv, "Ta": Ta, "Tb": Tb})
    features.to_csv(data_path + sub_ID + '/' + '/features.csv', index = False)
    fiducials_diag.to_csv(data_path + sub_ID + '/' + '/fiducials.csv', index = False)

7_GM


## Biomarkers extraction -- 24 Hours Diag Old Yes Int

In [12]:
import glob

In [13]:
HOM_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/HOM_mod/'
data_path = '/Volumes/Untitled/Hom3ostasis/CinC/Data_24hPulses/mod_oldQuality/'
sub_names = sorted([f for f in os.listdir(HOM_path) if not f.startswith('.')])
len(sub_names)

39

In [14]:
fs = 64
for i, sub_ID in enumerate(sub_names): 
    sub_ID = "12_FN"
    print(sub_ID)
    ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]
    # acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
    ppg_data = pd.read_csv(ppg_path, header = None)#[2:].to_numpy().squeeze()
    break
    # acc = pd.read_csv(acc_path, header = None)[2:].to_numpy().squeeze()
    y_basic = pd.read_csv(data_path + sub_ID + '/' + '/y_basic.csv', header = None).to_numpy().squeeze()
    y_diagnostic = pd.read_csv(data_path + sub_ID + '/' + '/y_diagnostic.csv', header = None).to_numpy().squeeze()
    I_foot = pd.read_csv(data_path + sub_ID + '/' + '/I_foot.csv', header=None).to_numpy().squeeze() - 1 # -1 because matlab is 1-indexed
    I_peak = pd.read_csv(data_path + sub_ID + '/' + '/I_peak.csv', header=None).to_numpy().squeeze() - 1
    if I_peak[0] < I_foot[0]:
        I_peak = I_peak[1:]
    if I_foot[-1] > I_peak[-1]:
        I_foot = I_foot[:-1]

    ppg_pulses = [ppg[I_foot[i]:I_foot[i+1]] for i in range(len(I_foot)-1)]

    # Interpolate each pulse to 100 samples with cubic spline
    ppg_pulses_interp = [np.interp(np.linspace(0, 1, 100), np.linspace(0, 1, len(pulse)), pulse) for pulse in ppg_pulses]

    # Reconstruct the overall ppg using the interpolated pulses
    I_foot = np.concatenate([[0], np.cumsum([len(pulse) for pulse in ppg_pulses_interp])])
    I_peak = np.array([np.argmax(ppg_pulses_interp[i]) + I_foot[i] for i in range(len(ppg_pulses_interp))])
    ppg = np.concatenate(ppg_pulses_interp)

    I_peak_diag = I_peak[y_diagnostic == 1]
    start_diag = I_foot[:-1][y_diagnostic == 1]
    end_diag = I_foot[1:][y_diagnostic == 1]
    diag_pulses = np.sort(np.concatenate((start_diag, end_diag)))
    diag_pulses[::2].shape, I_peak_diag.shape

    dn = get_dicrotic_notch(ppg, 64, I_peak, list(I_foot))
    dn_diag = np.array(dn)[y_diagnostic == 1]
    apg_fp = get_apg_fiducials(ppg, list(I_foot), I_peak)
    e_diag = apg_fp["e"][y_diagnostic == 1].values.astype(int)
    a_diag = apg_fp["a"][y_diagnostic == 1].values.astype(int)
    b_diag = apg_fp["b"][y_diagnostic == 1].values.astype(int)

    vpg_fp = get_vpg_fiducials(ppg, list(I_foot))
    u_diag = vpg_fp["u"][y_diagnostic == 1].values.astype(int)
    v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)
    dp_diag = get_diastolic_peak(ppg, diag_pulses, dn_diag, e_diag)
    fiducials_diag = pd.DataFrame({"onset": diag_pulses[::2], "offset": diag_pulses[1::2]-1, "sp": I_peak_diag, "dn": dn_diag, "dp": dp_diag, "u": u_diag, "v": v_diag, "a": a_diag, "b": b_diag})
    fiducials_diag[fiducials_diag < -5000000] = np.nan
    fiducials_diag.dropna(inplace=True)
    fiducials_diag = fiducials_diag.astype(int)

    # plt.figure()
    # plt.plot(ppg)
    # plt.plot(fiducials_diag["dp"], ppg[fiducials_diag["dp"]], 'k*', markersize=6)
    # plt.plot(fiducials_diag["dn"], ppg[fiducials_diag["dn"]], 'g*', markersize=6)
    # for j in range(y_diagnostic.shape[0]):
    #     if y_diagnostic[j] == 1:
    #         plt.plot(np.arange(I_foot[j],I_foot[j+1]), ppg[np.arange(I_foot[j],I_foot[j+1])], 'r')
    # ----- Time Intervals -----
    # Tpi: time between the pulse onset and offset
    # Tsys: time between the pulse onset and dn
    # Tdia: time between the dn and pulse offset
    # Tsp: time between the pulse onset and sp
    # Tdp: time between the pulse onset and dp
    # dT: time between sp and dp
    Tpi = fiducials_diag["offset"] - fiducials_diag["onset"]
    Tsys = fiducials_diag["dn"] - fiducials_diag["onset"]
    Tdia = fiducials_diag["offset"] - fiducials_diag["dn"]
    Tsp = fiducials_diag["sp"] - fiducials_diag["onset"]
    Tdp = fiducials_diag["dp"] - fiducials_diag["onset"]
    dT = fiducials_diag["dp"] - fiducials_diag["sp"]

    # ---- Amplitudes -----
    # Asp: difference in amplitude between onset and sp
    # Adn: difference in amplitude between onset and dn
    # Adp: difference in amplitude between onset and dp
    Asp = ppg[fiducials_diag["sp"]] - ppg[fiducials_diag["onset"]]
    Adn = ppg[fiducials_diag["dn"]] - ppg[fiducials_diag["onset"]]
    Adp = ppg[fiducials_diag["dp"]] - ppg[fiducials_diag["onset"]]

    # ---- Areas -----
    # AUCpi: area under the whole pulse (between onset and offset)
    # AUCsys: area under pulse wave between onset and dn
    # AUCdia: area under pulse wave between dn and offset
    AUCpi = np.zeros(len(fiducials_diag))
    AUCsys = np.zeros(len(fiducials_diag))
    AUCdia = np.zeros(len(fiducials_diag))
    for i in range(len(fiducials_diag)):
        AUCpi[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["offset"].iloc[i]])
        AUCsys[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["dn"].iloc[i]])
        AUCdia[i] = np.trapz(ppg[fiducials_diag["dn"].iloc[i]:fiducials_diag["offset"].iloc[i]])

    # ---- Ratios -----
    # TsysTdia: ratio between Tsys and Tdia
    # TspTpi: ratio between Tsp and Tpi
    # AdpAsp: ratio between Adp and Asp --> reflection index! 
    # IPA: ratio of the area under diastolic curve and the area under systolic curve
    # TspAsp: ratio between Tsp and Asp
    # AspdT: ratio between Asp and dT --> Stifness index
    # AspTpiTsp: ratio between Asp and (Tpi - Tsp)
    TsysTdia = Tsys/Tdia
    TspTpi = Tsp/Tpi
    AdpAsp = Adp/Asp
    IPA = AUCdia/AUCsys
    TspAsp = Tsp/Asp
    AspdT = Asp/dT
    AspTpiTsp = Asp/(Tpi - Tsp)
    Rslope = Asp/Tsys

    # ---- Derivatives -----
    # Tu: time between the pulse onset and u
    # Tv: time between the pulse onset and v
    # Ta: time between the pulse onset and a
    # Tb: time between the pulse onset and b
    Tu = fiducials_diag["u"] - fiducials_diag["onset"]
    Tv = fiducials_diag["v"] - fiducials_diag["onset"]
    Ta = fiducials_diag["a"] - fiducials_diag["onset"]
    Tb = fiducials_diag["b"] - fiducials_diag["onset"]

    features = pd.DataFrame({"Tpi": Tpi, "Tsys": Tsys, "Tdia": Tdia, "Tsp": Tsp, "Tdp": Tdp, "dT": dT, "Asp": Asp, "Adn": Adn, "Adp": Adp, "AUCpi": AUCpi, "AUCsys": AUCsys, "AUCdia": AUCdia, "Tsys/Tdia": TsysTdia, "Tsp/Tpi": TspTpi, "Adp/Asp": AdpAsp, "IPA": IPA, "Tsp/Asp": TspAsp, "Asp/dT": AspdT, "Asp/(Tpi-Tsp)": AspTpiTsp, "Rslope": Rslope, "Tu": Tu, "Tv": Tv, "Ta": Ta, "Tb": Tb})
    features.to_csv(data_path + sub_ID + '/' + '/features.csv', index = False)
    fiducials_diag.to_csv(data_path + sub_ID + '/' + '/fiducials.csv', index = False)

12_FN


In [12]:
pd.to_datetime(ppg.iloc[0], unit='s')

0   2019-11-27 09:27:00
Name: 0, dtype: datetime64[ns]

## Biomarkers extraction -- Baseline

In [3]:
import glob
import os

In [20]:
HOM_path = "/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/Data/baselineExcept1st5/"
data_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Data_24hPulses/baseline/'
sub_names = sorted([f for f in os.listdir(HOM_path) if not f.startswith('.')])
len(sub_names)

39

In [15]:
plt.figure()
plt.plot(ppg)

In [15]:
output_path = "/Volumes/Untitled/Hom3ostasis/CinC/Data_lab/baseline/"

0    1.573819e+09
Name: 1, dtype: float64

In [25]:
fs = 64
for i, sub_ID in enumerate(sub_names): 
    print(sub_ID)
    ppg_path = HOM_path + sub_ID + "/Standard/ppg_data.csv"
    # acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
    ppg = pd.read_csv(ppg_path, header = None)#[2:].to_numpy().squeeze()

    # acc = pd.read_csv(acc_path, header = None)[2:].to_numpy().squeeze()
    y_basic = pd.read_csv(HOM_path + sub_ID + "/Standard/y_basic.csv", header = None).to_numpy().squeeze()
    y_diagnostic = pd.read_csv(HOM_path + sub_ID + "/Standard/y_diagnostic.csv", header = None).to_numpy().squeeze()
    I_foot = pd.read_csv(HOM_path + sub_ID + "/Standard/I_foot.csv", header=None).to_numpy().squeeze() - 1 # -1 because matlab is 1-indexed
    I_peak = pd.read_csv(HOM_path + sub_ID + "/Standard/I_peak.csv", header=None).to_numpy().squeeze() - 1

    if I_peak[0] < I_foot[0]:
        I_peak = I_peak[1:]
    if I_foot[-1] > I_peak[-1]:
        I_foot = I_foot[:-1]

    ppg_pulses = [ppg[I_foot[i]:I_foot[i+1]] for i in range(len(I_foot)-1)]

    # Interpolate each pulse to 100 samples with cubic spline
    ppg_pulses_interp = [np.interp(np.linspace(0, 1, 100), np.linspace(0, 1, len(pulse)), pulse) for pulse in ppg_pulses]

    # Reconstruct the overall ppg using the interpolated pulses
    I_foot = np.concatenate([[0], np.cumsum([len(pulse) for pulse in ppg_pulses_interp])])
    I_peak = np.array([np.argmax(ppg_pulses_interp[i]) + I_foot[i] for i in range(len(ppg_pulses_interp))])
    ppg = np.concatenate(ppg_pulses_interp)

    I_peak_diag = I_peak[y_basic == 1]
    start_diag = I_foot[:-1][y_basic == 1]
    end_diag = I_foot[1:][y_basic == 1]
    diag_pulses = np.sort(np.concatenate((start_diag, end_diag)))
    diag_pulses[::2].shape, I_peak_diag.shape

    dn = get_dicrotic_notch(ppg, 64, I_peak, list(I_foot))
    dn_diag = np.array(dn)[y_basic == 1]
    apg_fp = get_apg_fiducials(ppg, list(I_foot), I_peak)
    e_diag = apg_fp["e"][y_basic == 1].values.astype(int)
    a_diag = apg_fp["a"][y_basic == 1].values.astype(int)
    b_diag = apg_fp["b"][y_basic == 1].values.astype(int)

    vpg_fp = get_vpg_fiducials(ppg, list(I_foot))
    u_diag = vpg_fp["u"][y_basic == 1].values.astype(int)
    v_diag = vpg_fp["v"][y_basic == 1].values.astype(int)
    dp_diag = get_diastolic_peak(ppg, diag_pulses, dn_diag, e_diag)
    fiducials_diag = pd.DataFrame({"onset": diag_pulses[::2], "offset": diag_pulses[1::2]-1, "sp": I_peak_diag, "dn": dn_diag, "dp": dp_diag, "u": u_diag, "v": v_diag, "a": a_diag, "b": b_diag})
    fiducials_diag[fiducials_diag < -5000000] = np.nan
    fiducials_diag.dropna(inplace=True)
    fiducials_diag = fiducials_diag.astype(int)

    # plt.figure()
    # plt.plot(ppg)
    # plt.plot(fiducials_diag["dp"], ppg[fiducials_diag["dp"]], 'k*', markersize=6)
    # plt.plot(fiducials_diag["dn"], ppg[fiducials_diag["dn"]], 'g*', markersize=6)
    # for j in range(y_basic.shape[0]):
    #     if y_basic[j] == 1:
    #         plt.plot(np.arange(I_foot[j],I_foot[j+1]), ppg[np.arange(I_foot[j],I_foot[j+1])], 'r')

    # break
    # ----- Time Intervals -----
    # Tpi: time between the pulse onset and offset
    # Tsys: time between the pulse onset and dn
    # Tdia: time between the dn and pulse offset
    # Tsp: time between the pulse onset and sp
    # Tdp: time between the pulse onset and dp
    # dT: time between sp and dp
    Tpi = fiducials_diag["offset"] - fiducials_diag["onset"]
    Tsys = fiducials_diag["dn"] - fiducials_diag["onset"]
    Tdia = fiducials_diag["offset"] - fiducials_diag["dn"]
    Tsp = fiducials_diag["sp"] - fiducials_diag["onset"]
    Tdp = fiducials_diag["dp"] - fiducials_diag["onset"]
    dT = fiducials_diag["dp"] - fiducials_diag["sp"]

    # ---- Amplitudes -----
    # Asp: difference in amplitude between onset and sp
    # Adn: difference in amplitude between onset and dn
    # Adp: difference in amplitude between onset and dp
    Asp = ppg[fiducials_diag["sp"]] - ppg[fiducials_diag["onset"]]
    Adn = ppg[fiducials_diag["dn"]] - ppg[fiducials_diag["onset"]]
    Adp = ppg[fiducials_diag["dp"]] - ppg[fiducials_diag["onset"]]

    # ---- Areas -----
    # AUCpi: area under the whole pulse (between onset and offset)
    # AUCsys: area under pulse wave between onset and dn
    # AUCdia: area under pulse wave between dn and offset
    AUCpi = np.zeros(len(fiducials_diag))
    AUCsys = np.zeros(len(fiducials_diag))
    AUCdia = np.zeros(len(fiducials_diag))
    for i in range(len(fiducials_diag)):
        AUCpi[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["offset"].iloc[i]])
        AUCsys[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["dn"].iloc[i]])
        AUCdia[i] = np.trapz(ppg[fiducials_diag["dn"].iloc[i]:fiducials_diag["offset"].iloc[i]])

    # ---- Ratios -----
    # TsysTdia: ratio between Tsys and Tdia
    # TspTpi: ratio between Tsp and Tpi
    # AdpAsp: ratio between Adp and Asp --> reflection index! 
    # IPA: ratio of the area under diastolic curve and the area under systolic curve
    # TspAsp: ratio between Tsp and Asp
    # AspdT: ratio between Asp and dT --> Stiffness index
    # AspTpiTsp: ratio between Asp and (Tpi - Tsp)
    # Rslope: ratio between Asp and Tsys
    TsysTdia = Tsys/Tdia
    TspTpi = Tsp/Tpi
    AdpAsp = Adp/Asp
    IPA = AUCdia/AUCsys
    TspAsp = Tsp/Asp
    AspdT = Asp/dT
    AspTpiTsp = Asp/(Tpi - Tsp)
    Rslope = Asp/Tsys

    # ---- Derivatives -----
    # Tu: time between the pulse onset and u
    # Tv: time between the pulse onset and v
    # Ta: time between the pulse onset and a
    # Tb: time between the pulse onset and b
    Tu = fiducials_diag["u"] - fiducials_diag["onset"]
    Tv = fiducials_diag["v"] - fiducials_diag["onset"]
    Ta = fiducials_diag["a"] - fiducials_diag["onset"]
    Tb = fiducials_diag["b"] - fiducials_diag["onset"]

    features = pd.DataFrame({"Tpi": Tpi, "Tsys": Tsys, "Tdia": Tdia, "Tsp": Tsp, "Tdp": Tdp, "dT": dT, "Asp": Asp, "Adn": Adn, "Adp": Adp, "AUCpi": AUCpi, "AUCsys": AUCsys, "AUCdia": AUCdia, "Tsys/Tdia": TsysTdia, "Tsp/Tpi": TspTpi, "Adp/Asp": AdpAsp, "IPA": IPA, "Tsp/Asp": TspAsp, "Asp/dT": AspdT, "Asp/(Tpi-Tsp)": AspTpiTsp, "Rslope": Rslope, "Tu": Tu, "Tv": Tv, "Ta": Ta, "Tb": Tb})
    features.to_csv(output_path + sub_ID + '/' + '/features_norm.csv', index = False)
    fiducials_diag.to_csv(output_path + sub_ID + '/' + '/fiducials_norm.csv', index = False)

10_AC
0   2019-11-19 10:21:43.059999943
Name: 1, dtype: datetime64[ns]
11_EC
0   2019-11-21 10:23:07.529999970
Name: 1, dtype: datetime64[ns]
12_FN
0   2019-11-27 09:27:54.920000076
Name: 1, dtype: datetime64[ns]
13_MS
0   2019-11-28 10:13:23.109999895
Name: 1, dtype: datetime64[ns]
14_AD
0   2019-12-06 09:23:03.900000095
Name: 1, dtype: datetime64[ns]
15_ER
0   2019-12-06 10:19:40.829999924
Name: 1, dtype: datetime64[ns]
16_PD
0   2019-12-11 11:45:56.299999952
Name: 1, dtype: datetime64[ns]
17_EG
0   2019-12-12 10:14:21.410000086
Name: 1, dtype: datetime64[ns]
18_MV
0   2019-12-12 11:16:52.049999952
Name: 1, dtype: datetime64[ns]
19_FI
0   2019-12-18 11:18:23.900000095
Name: 1, dtype: datetime64[ns]
1_ID
0   2021-09-30 07:35:28.220000029
Name: 1, dtype: datetime64[ns]
20_AC
0   2019-12-19 11:24:13.650000095
Name: 1, dtype: datetime64[ns]
21_CK
0   2020-07-23 09:25:56.859999895
Name: 1, dtype: datetime64[ns]
22_CM
0   2020-08-05 09:03:49.950000048
Name: 1, dtype: datetime64[ns]
23_PP
0

In [8]:
I_peak.shape, I_foot.shape, y_basic.shape

((215,), (216,), (215,))

In [ ]:
HOM_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/HOM2/'
data_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/NewAnalysis2023/CinC/Data_24hPulses/followup/'
sub_names = sorted([f for f in os.listdir(data_path) if not f.startswith('.')])
len(sub_names)

39

In [ ]:
fs = 64
for i, sub_ID in enumerate(sub_names): 
    print(sub_ID)
    ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]
    acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
    ppg = pd.read_csv(ppg_path, header = None)[2:].to_numpy().squeeze()
    acc = pd.read_csv(acc_path, header = None)[2:].to_numpy().squeeze()
    y_basic = pd.read_csv(data_path + sub_ID + '/' + '/y_basic.csv', header = None).to_numpy().squeeze()
    y_diagnostic = pd.read_csv(data_path + sub_ID + '/' + '/y_diagnostic.csv', header = None).to_numpy().squeeze()
    I_foot = pd.read_csv(data_path + sub_ID + '/' + '/I_foot.csv', header=None).to_numpy().squeeze() - 1 # -1 because matlab is 1-indexed
    I_peak = pd.read_csv(data_path + sub_ID + '/' + '/I_peak.csv', header=None).to_numpy().squeeze() - 1
    if I_peak[0] < I_foot[0]:
        I_peak = I_peak[1:]
    if I_foot[-1] > I_peak[-1]:
        I_foot = I_foot[:-1]
    I_peak_diag = I_peak[:-1][y_diagnostic == 1]
    start_diag = I_foot[:-1][y_diagnostic == 1]
    end_diag = I_foot[1:][y_diagnostic == 1]
    diag_pulses = np.sort(np.concatenate((start_diag, end_diag)))
    diag_pulses[::2].shape, I_peak_diag.shape

    dn = get_dicrotic_notch(ppg, 64, I_peak, list(I_foot))
    dn_diag = np.array(dn)[y_diagnostic == 1]
    apg_fp = get_apg_fiducials(ppg, list(I_foot), I_peak)
    e_diag = apg_fp["e"][y_diagnostic == 1].values.astype(int)
    a_diag = apg_fp["a"][y_diagnostic == 1].values.astype(int)
    b_diag = apg_fp["b"][y_diagnostic == 1].values.astype(int)

    vpg_fp = get_vpg_fiducials(ppg, list(I_foot))
    u_diag = vpg_fp["u"][y_diagnostic == 1].values.astype(int)
    v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)
    dp_diag = get_diastolic_peak(ppg, diag_pulses, dn_diag, e_diag)
    fiducials_diag = pd.DataFrame({"onset": diag_pulses[::2], "offset": diag_pulses[1::2]-1, "sp": I_peak_diag, "dn": dn_diag, "dp": dp_diag, "u": u_diag, "v": v_diag, "a": a_diag, "b": b_diag})
    fiducials_diag[fiducials_diag < -5000000] = np.nan
    fiducials_diag.dropna(inplace=True)
    fiducials_diag = fiducials_diag.astype(int)

    # plt.figure()
    # plt.plot(ppg)
    # plt.plot(fiducials_diag["dp"], ppg[fiducials_diag["dp"]], 'k*', markersize=6)
    # plt.plot(fiducials_diag["dn"], ppg[fiducials_diag["dn"]], 'g*', markersize=6)
    # for j in range(y_diagnostic.shape[0]):
    #     if y_diagnostic[j] == 1:
    #         plt.plot(np.arange(I_foot[j],I_foot[j+1]), ppg[np.arange(I_foot[j],I_foot[j+1])], 'r')
    # ----- Time Intervals -----
    # Tpi: time between the pulse onset and offset
    # Tsys: time between the pulse onset and dn
    # Tdia: time between the dn and pulse offset
    # Tsp: time between the pulse onset and sp
    # Tdp: time between the pulse onset and dp
    # dT: time between sp and dp
    Tpi = fiducials_diag["offset"]/fs - fiducials_diag["onset"]/fs
    Tsys = fiducials_diag["dn"]/fs - fiducials_diag["onset"]/fs
    Tdia = fiducials_diag["offset"]/fs - fiducials_diag["dn"]/fs
    Tsp = fiducials_diag["sp"]/fs - fiducials_diag["onset"]/fs
    Tdp = fiducials_diag["dp"]/fs - fiducials_diag["onset"]/fs
    dT = fiducials_diag["dp"]/fs - fiducials_diag["sp"]/fs

    # ---- Amplitudes -----
    # Asp: difference in amplitude between onset and sp
    # Adn: difference in amplitude between onset and dn
    # Adp: difference in amplitude between onset and dp
    Asp = ppg[fiducials_diag["sp"]] - ppg[fiducials_diag["onset"]]
    Adn = ppg[fiducials_diag["dn"]] - ppg[fiducials_diag["onset"]]
    Adp = ppg[fiducials_diag["dp"]] - ppg[fiducials_diag["onset"]]

    # ---- Areas -----
    # AUCpi: area under the whole pulse (between onset and offset)
    # AUCsys: area under pulse wave between onset and dn
    # AUCdia: area under pulse wave between dn and offset
    AUCpi = np.zeros(len(fiducials_diag))
    AUCsys = np.zeros(len(fiducials_diag))
    AUCdia = np.zeros(len(fiducials_diag))
    for i in range(len(fiducials_diag)):
        AUCpi[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["offset"].iloc[i]])
        AUCsys[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["dn"].iloc[i]])
        AUCdia[i] = np.trapz(ppg[fiducials_diag["dn"].iloc[i]:fiducials_diag["offset"].iloc[i]])

    # ---- Ratios -----
    # TsysTdia: ratio between Tsys and Tdia
    # TspTpi: ratio between Tsp and Tpi
    # AdpAsp: ratio between Adp and Asp --> reflection index! 
    # IPA: ratio of the area under diastolic curve and the area under systolic curve
    # TspAsp: ratio between Tsp and Asp
    # AspdT: ratio between Asp and dT --> Stifness index
    # AspTpiTsp: ratio between Asp and (Tpi - Tsp)
    TsysTdia = Tsys/Tdia
    TspTpi = Tsp/Tpi
    AdpAsp = Adp/Asp
    IPA = AUCdia/AUCsys
    TspAsp = Tsp/Asp
    AspdT = Asp/dT
    AspTpiTsp = Asp/(Tpi - Tsp)

    # ---- Derivatives -----
    # Tu: time between the pulse onset and u
    # Tv: time between the pulse onset and v
    # Ta: time between the pulse onset and a
    # Tb: time between the pulse onset and b
    Tu = fiducials_diag["u"]/fs - fiducials_diag["onset"]/fs
    Tv = fiducials_diag["v"]/fs - fiducials_diag["onset"]/fs
    Ta = fiducials_diag["a"]/fs - fiducials_diag["onset"]/fs
    Tb = fiducials_diag["b"]/fs - fiducials_diag["onset"]/fs

    features = pd.DataFrame({"Tpi": Tpi, "Tsys": Tsys, "Tdia": Tdia, "Tsp": Tsp, "Tdp": Tdp, "dT": dT, "Asp": Asp, "Adn": Adn, "Adp": Adp, "AUCpi": AUCpi, "AUCsys": AUCsys, "AUCdia": AUCdia, "Tsys/Tdia": TsysTdia, "Tsp/Tpi": TspTpi, "Adp/Asp": AdpAsp, "IPA": IPA, "Tsp/Asp": TspAsp, "Asp/dT": AspdT, "Asp/(Tpi-Tsp)": AspTpiTsp, "Tu": Tu, "Tv": Tv, "Ta": Ta, "Tb": Tb})
    features.to_csv(data_path + sub_ID + '/' + '/features.csv', index = False)
    fiducials_diag.to_csv(data_path + sub_ID + '/' + '/fiducials.csv', index = False)

10_AC
11_EC
12_FN


/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:25: RuntimeWarning: invalid value encountered in cast
  e_diag = apg_fp["e"][y_diagnostic == 1].values.astype(int)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:26: RuntimeWarning: invalid value encountered in cast
  a_diag = apg_fp["a"][y_diagnostic == 1].values.astype(int)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:27: RuntimeWarning: invalid value encountered in cast
  b_diag = apg_fp["b"][y_diagnostic == 1].values.astype(int)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:31: RuntimeWarning: invalid value encountered in cast
  v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)


13_MS


/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:31: RuntimeWarning: invalid value encountered in cast
  v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)


14_AD
15_ER


...

/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:31: RuntimeWarning: invalid value encountered in cast
  v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)


8_MB


/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:31: RuntimeWarning: invalid value encountered in cast
  v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)


9_BC


/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:25: RuntimeWarning: invalid value encountered in cast
  e_diag = apg_fp["e"][y_diagnostic == 1].values.astype(int)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:26: RuntimeWarning: invalid value encountered in cast
  a_diag = apg_fp["a"][y_diagnostic == 1].values.astype(int)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:27: RuntimeWarning: invalid value encountered in cast
  b_diag = apg_fp["b"][y_diagnostic == 1].values.astype(int)
/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_56585/1758887707.py:31: RuntimeWarning: invalid value encountered in cast
  v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)


In [ ]:
plt.close("all")

In [ ]:
fiducials_diag[fiducials_diag < -5000000] = np.nan
fiducials_diag.dropna(inplace=True)

## Extract other features in 2.30 min windows

In [5]:
import glob

In [6]:
HOM_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/HOM_mod/'

In [8]:
sub_ID = "12_FN"
print(sub_ID)
ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]
# acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
ppg_data = pd.read_csv(ppg_path, header = None)#[2:].to_numpy().squeeze()
ppg_start = pd.to_datetime(ppg_data.iloc[0], unit='s')
ppg = ppg_data[2:].to_numpy().squeeze()
t_ppg = pd.date_range(start=ppg_start.iloc[0], periods = len(ppg), freq = '0.015625s')

12_FN


In [9]:
ppg_df = pd.Series(ppg, index = t_ppg)

ppg_df.head()

2019-11-27 09:27:00.000000   -0.0
2019-11-27 09:27:00.015625   -0.0
2019-11-27 09:27:00.031250   -0.0
2019-11-27 09:27:00.046875   -0.0
2019-11-27 09:27:00.062500   -0.0
Freq: 15625us, dtype: float64

In [11]:
plt.figure()
plt.plot(ppg_df)

In [15]:
pd.to_datetime(1573819077.000000, unit = "s")

Timestamp('2019-11-15 11:57:57')

In [12]:
sampling_rate = 64  # Example sampling rate in Hz
window_duration = 150  # Window duration in seconds

# Calculate the number of data points in each window
window_size = window_duration * sampling_rate

# Calculate the total number of windows possible
total_windows = len(ppg) // window_size

# List to hold each window of data
windows = []

# Extract each window of data
for i in range(total_windows):
    start_index = i * window_size
    end_index = start_index + window_size
    window = ppg[start_index:end_index]
    windows.append(window)

In [14]:
plt.plot(windows[222])

In [ ]:
import glob
HOM_path = '/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Acq/HOM_mod/'
data_path = '/Volumes/Untitled/Hom3ostasis/CinC/Data_24hPulses/mod_oldQuality/'
sub_names = sorted([f for f in os.listdir(HOM_path) if not f.startswith('.')])
len(sub_names)
fs = 64
for i, sub_ID in enumerate(sub_names): 
    sub_ID = "12_FN"
    print(sub_ID)
    ppg_path = glob.glob(HOM_path + sub_ID + '/**/BVP.csv', recursive=True)[0]
    # acc_path = glob.glob(HOM_path + sub_ID + '/**/ACC.csv', recursive=True)[0]
    ppg_data = pd.read_csv(ppg_path, header = None)#[2:].to_numpy().squeeze()
    ppg_start = pd.to_datetime(ppg_data.iloc[0], unit='s')
    t_ppg = pd.date_range(start=ppg_start, periods=ppg_data.shape[0], freq='S')
    ppg = ppg_data[2:].to_numpy().squeeze()
    break
    # acc = pd.read_csv(acc_path, header = None)[2:].to_numpy().squeeze()
    y_basic = pd.read_csv(data_path + sub_ID + '/' + '/y_basic.csv', header = None).to_numpy().squeeze()
    y_diagnostic = pd.read_csv(data_path + sub_ID + '/' + '/y_diagnostic.csv', header = None).to_numpy().squeeze()
    I_foot = pd.read_csv(data_path + sub_ID + '/' + '/I_foot.csv', header=None).to_numpy().squeeze() - 1 # -1 because matlab is 1-indexed
    I_peak = pd.read_csv(data_path + sub_ID + '/' + '/I_peak.csv', header=None).to_numpy().squeeze() - 1
    if I_peak[0] < I_foot[0]:
        I_peak = I_peak[1:]
    if I_foot[-1] > I_peak[-1]:
        I_foot = I_foot[:-1]

    ppg_pulses = [ppg[I_foot[i]:I_foot[i+1]] for i in range(len(I_foot)-1)]

    # Interpolate each pulse to 100 samples with cubic spline
    ppg_pulses_interp = [np.interp(np.linspace(0, 1, 100), np.linspace(0, 1, len(pulse)), pulse) for pulse in ppg_pulses]

    # Reconstruct the overall ppg using the interpolated pulses
    I_foot = np.concatenate([[0], np.cumsum([len(pulse) for pulse in ppg_pulses_interp])])
    I_peak = np.array([np.argmax(ppg_pulses_interp[i]) + I_foot[i] for i in range(len(ppg_pulses_interp))])
    ppg = np.concatenate(ppg_pulses_interp)

    I_peak_diag = I_peak[y_diagnostic == 1]
    start_diag = I_foot[:-1][y_diagnostic == 1]
    end_diag = I_foot[1:][y_diagnostic == 1]
    diag_pulses = np.sort(np.concatenate((start_diag, end_diag)))
    diag_pulses[::2].shape, I_peak_diag.shape

    dn = get_dicrotic_notch(ppg, 64, I_peak, list(I_foot))
    dn_diag = np.array(dn)[y_diagnostic == 1]
    apg_fp = get_apg_fiducials(ppg, list(I_foot), I_peak)
    e_diag = apg_fp["e"][y_diagnostic == 1].values.astype(int)
    a_diag = apg_fp["a"][y_diagnostic == 1].values.astype(int)
    b_diag = apg_fp["b"][y_diagnostic == 1].values.astype(int)

    vpg_fp = get_vpg_fiducials(ppg, list(I_foot))
    u_diag = vpg_fp["u"][y_diagnostic == 1].values.astype(int)
    v_diag = vpg_fp["v"][y_diagnostic == 1].values.astype(int)
    dp_diag = get_diastolic_peak(ppg, diag_pulses, dn_diag, e_diag)
    fiducials_diag = pd.DataFrame({"onset": diag_pulses[::2], "offset": diag_pulses[1::2]-1, "sp": I_peak_diag, "dn": dn_diag, "dp": dp_diag, "u": u_diag, "v": v_diag, "a": a_diag, "b": b_diag})
    fiducials_diag[fiducials_diag < -5000000] = np.nan
    fiducials_diag.dropna(inplace=True)
    fiducials_diag = fiducials_diag.astype(int)

    # plt.figure()
    # plt.plot(ppg)
    # plt.plot(fiducials_diag["dp"], ppg[fiducials_diag["dp"]], 'k*', markersize=6)
    # plt.plot(fiducials_diag["dn"], ppg[fiducials_diag["dn"]], 'g*', markersize=6)
    # for j in range(y_diagnostic.shape[0]):
    #     if y_diagnostic[j] == 1:
    #         plt.plot(np.arange(I_foot[j],I_foot[j+1]), ppg[np.arange(I_foot[j],I_foot[j+1])], 'r')
    # ----- Time Intervals -----
    # Tpi: time between the pulse onset and offset
    # Tsys: time between the pulse onset and dn
    # Tdia: time between the dn and pulse offset
    # Tsp: time between the pulse onset and sp
    # Tdp: time between the pulse onset and dp
    # dT: time between sp and dp
    Tpi = fiducials_diag["offset"] - fiducials_diag["onset"]
    Tsys = fiducials_diag["dn"] - fiducials_diag["onset"]
    Tdia = fiducials_diag["offset"] - fiducials_diag["dn"]
    Tsp = fiducials_diag["sp"] - fiducials_diag["onset"]
    Tdp = fiducials_diag["dp"] - fiducials_diag["onset"]
    dT = fiducials_diag["dp"] - fiducials_diag["sp"]

    # ---- Amplitudes -----
    # Asp: difference in amplitude between onset and sp
    # Adn: difference in amplitude between onset and dn
    # Adp: difference in amplitude between onset and dp
    Asp = ppg[fiducials_diag["sp"]] - ppg[fiducials_diag["onset"]]
    Adn = ppg[fiducials_diag["dn"]] - ppg[fiducials_diag["onset"]]
    Adp = ppg[fiducials_diag["dp"]] - ppg[fiducials_diag["onset"]]

    # ---- Areas -----
    # AUCpi: area under the whole pulse (between onset and offset)
    # AUCsys: area under pulse wave between onset and dn
    # AUCdia: area under pulse wave between dn and offset
    AUCpi = np.zeros(len(fiducials_diag))
    AUCsys = np.zeros(len(fiducials_diag))
    AUCdia = np.zeros(len(fiducials_diag))
    for i in range(len(fiducials_diag)):
        AUCpi[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["offset"].iloc[i]])
        AUCsys[i] = np.trapz(ppg[fiducials_diag["onset"].iloc[i]:fiducials_diag["dn"].iloc[i]])
        AUCdia[i] = np.trapz(ppg[fiducials_diag["dn"].iloc[i]:fiducials_diag["offset"].iloc[i]])

    # ---- Ratios -----
    # TsysTdia: ratio between Tsys and Tdia
    # TspTpi: ratio between Tsp and Tpi
    # AdpAsp: ratio between Adp and Asp --> reflection index! 
    # IPA: ratio of the area under diastolic curve and the area under systolic curve
    # TspAsp: ratio between Tsp and Asp
    # AspdT: ratio between Asp and dT --> Stifness index
    # AspTpiTsp: ratio between Asp and (Tpi - Tsp)
    TsysTdia = Tsys/Tdia
    TspTpi = Tsp/Tpi
    AdpAsp = Adp/Asp
    IPA = AUCdia/AUCsys
    TspAsp = Tsp/Asp
    AspdT = Asp/dT
    AspTpiTsp = Asp/(Tpi - Tsp)
    Rslope = Asp/Tsys

    # ---- Derivatives -----
    # Tu: time between the pulse onset and u
    # Tv: time between the pulse onset and v
    # Ta: time between the pulse onset and a
    # Tb: time between the pulse onset and b
    Tu = fiducials_diag["u"] - fiducials_diag["onset"]
    Tv = fiducials_diag["v"] - fiducials_diag["onset"]
    Ta = fiducials_diag["a"] - fiducials_diag["onset"]
    Tb = fiducials_diag["b"] - fiducials_diag["onset"]

    features = pd.DataFrame({"Tpi": Tpi, "Tsys": Tsys, "Tdia": Tdia, "Tsp": Tsp, "Tdp": Tdp, "dT": dT, "Asp": Asp, "Adn": Adn, "Adp": Adp, "AUCpi": AUCpi, "AUCsys": AUCsys, "AUCdia": AUCdia, "Tsys/Tdia": TsysTdia, "Tsp/Tpi": TspTpi, "Adp/Asp": AdpAsp, "IPA": IPA, "Tsp/Asp": TspAsp, "Asp/dT": AspdT, "Asp/(Tpi-Tsp)": AspTpiTsp, "Rslope": Rslope, "Tu": Tu, "Tv": Tv, "Ta": Ta, "Tb": Tb})
    features.to_csv(data_path + sub_ID + '/' + '/features.csv', index = False)
    fiducials_diag.to_csv(data_path + sub_ID + '/' + '/fiducials.csv', index = False)